# GRCh38 DNase 12 kb CAE Optimizer Search 2

**Tasks:** evaluate if momentum is the reason why Adam fails

**Insights:**

- 

In [15]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

import json
import numpy as np
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
testing = False
    
# The base directory is one level up
base = '..'
name = 'optimizer-search-2'
dataset = 'cnn-search'
settings_filepath = '../settings-grch38-chip-12kb.json'
search_filepath = '../{}.json'.format(name)

with open(settings_filepath, "r") as f:
    settings = json.load(f)

with open(search_filepath, "r") as f:
    search = json.load(f)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create training jobs

In [16]:
from jobs import jobs

jobs(
    os.path.relpath(search_filepath, base),
    os.path.relpath(settings_filepath, base),
    dataset=dataset,
    name=name,
    cluster='cox',
    epochs=25,
    batch_size=256,
    base=base,
    clear=True,
    verbose=False
)

Created slurm file for training 65 neural networks


## Test training run of the most complex CAE

In [12]:
from train import train_on_single_dataset

model_name = 'cf-32-64-128-256--ck-3-7-11-15--du-512-128--do-0-0-0-0-0-0--e-8--rl-0--o-adam-0.2-0.99--lr-0.001--lrd-0.001--l-smse-10--m---bn-0-0-0-0-0-0--bni-0--pw-2--sw-logn--swz-0.02'

with open('../models/{}.json'.format(model_name), 'r') as f:
    definition = json.load(f)

train_on_single_dataset(
    settings,
    dataset,
    definition,
    epochs=2,
    batch_size=256,
    peak_weight=2,
    signal_weighting='logn',
    signal_weighting_zero_point_percentage=0.02,
    base=base,
    clear=True,
)

2 logn 0.02


KeyboardInterrupt: 

In [ ]:
from ae.utils import check_status

okay, not_found, outdated = check_status(
    name, 'training', 'cnn-search', base=base
)

if okay:
    print('Trainings completed')
else:
    print('Trainings did not finish. Missing {}'.format(len(not_found))

## Create evaluation jobs

In [ ]:
from evaluate import create_jobs

create_jobs(
    name,
    name=name,
    dataset=dataset,
    cluster='seas',
    base=base,
    clear=True,
    incl_dtw=False,
)

In a terminal run: `sbatch evaluate-optimizer-search-2.slurm`

In [ ]:
from ae.utils import check_status

okay, not_found, outdated = check_status(
    name, 'evaluate', 'cnn-search', base=base
)

if okay:
    print('Evaluation completed')
else:
    print('Evaluation did not finish. Missing {}'.format(len(not_found))

## Compare

In [ ]:
from compare import compare
    
performance = compare(
    'definitions-{}.json'.format(name),
    dataset_name=dataset,
    base=base,
    clear=False,
    verbose=False,
    silent=False,
    remove_common_prefix_from_df=True,
)

In [ ]:
import qgrid

qgw = qgrid.show_grid(performance)
qgw

In [ ]:
from IPython.core.display import Image, display

for model in qgw.get_selected_df().iterrows():
    print(model[0])
    display(Image(os.path.join(base, 'models', 'cf-{}---predictions-{}.png'.format(model[0], dataset))))